# Intermediate Tutorial: Choosing the Right Optimizer

This tutorial explores different optimization methods and when to use them. We'll work with increasingly complex objective function surfaces to understand why different algorithms are needed.

## Learning Objectives

By the end of this tutorial, you will:

- Understand when to use different optimization methods
- See how complex objective surfaces affect optimization
- Learn about global vs local optimizers
- Compare results from different methods


## 1. Setup


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from ezfit.examples import (
    generate_gaussian_data,
    generate_linear_data,
    generate_multi_peak_data,
    generate_rugged_surface_data,
)

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 8)

## 2. Simple Case: Linear Fitting

For simple, well-behaved functions, the default `curve_fit` method works perfectly:


In [ ]:
# Generate simple linear data
df_linear = generate_linear_data(n_points=50, slope=2.0, intercept=1.0, seed=42)

def line(x, m, b):
    return m * x + b

# Default method: curve_fit (Levenberg-Marquardt)
model1, ax1, _ = df_linear.fit(line, "x", "y", "yerr", method="curve_fit")
plt.show()
print("Default method (curve_fit):")
print(f"  m = {model1['m'].value:.4f} ± {model1['m'].err:.4f}")
print(f"  b = {model1['b'].value:.4f} ± {model1['b'].err:.4f}")
print(f"  χ² = {model1.𝜒2:.2f}")

## 3. When Initial Guesses Matter: Gaussian Peak

For functions with multiple parameters or local minima, initial guesses become important. Let's fit a Gaussian peak:


In [ ]:
# Generate Gaussian peak data
df_gauss = generate_gaussian_data(
    n_points=100,
    amplitude=10.0,
    center=5.0,
    fwhm=2.0,
    seed=42
)

# Use built-in Gaussian function
from ezfit import gaussian

# Try with poor initial guess
print("=== Poor Initial Guess ===")
model_bad, ax_bad, _ = df_gauss.fit(
    gaussian, "x", "y", "yerr",
    amplitude={"value": 5.0},  # Too low
    center={"value": 3.0},     # Wrong position
    fwhm={"value": 1.0}        # Too narrow
)
plt.show()
print(model_bad)

# Try with good initial guess
print("\n=== Good Initial Guess ===")
model_good, ax_good, _ = df_gauss.fit(
    gaussian, "x", "y", "yerr",
    amplitude={"value": 9.0},  # Close to true value
    center={"value": 5.0},     # Close to true value
    fwhm={"value": 2.0}        # Close to true value
)
plt.show()
print(model_good)

## 4. Global Optimization: Rugged Surface

When the objective function has multiple local minima, local optimizers can get stuck. This is when you need global optimizers like `differential_evolution`:


In [ ]:
# Generate data with complex, multi-modal surface
df_rugged = generate_rugged_surface_data(n_points=100, seed=42)

# Define a model that approximates the complex function
# y = A*sin(x)*exp(-x/B) + C*sin(D*x) + E
def complex_model(x, A, B, C, D, E):
    """Complex model with multiple local minima"""
    return A * np.sin(x) * np.exp(-x / B) + C * np.sin(D * x) + E

# Try with local optimizer (may get stuck in local minimum)
print("=== Local Optimizer (minimize) ===")
try:
    model_local, ax_local, _ = df_rugged.fit(
        complex_model, "x", "y", "yerr",
        method="minimize",
        fit_kwargs={"method": "L-BFGS-B"},
        A={"value": 1.0, "min": -5, "max": 5},
        B={"value": 5.0, "min": 1, "max": 10},
        C={"value": 0.5, "min": -2, "max": 2},
        D={"value": 3.0, "min": 1, "max": 5},
        E={"value": 2.0, "min": 0, "max": 5}
    )
    plt.show()
    print(f"χ² = {model_local.𝜒2:.2f}")
except Exception as e:
    print(f"Failed: {e}")

# Try with global optimizer
print("\n=== Global Optimizer (differential_evolution) ===")
model_global, ax_global, _ = df_rugged.fit(
    complex_model, "x", "y", "yerr",
    method="differential_evolution",
    fit_kwargs={"maxiter": 1000, "seed": 42},
    A={"value": 1.0, "min": -5, "max": 5},
    B={"value": 5.0, "min": 1, "max": 10},
    C={"value": 0.5, "min": -2, "max": 2},
    D={"value": 3.0, "min": 1, "max": 5},
    E={"value": 2.0, "min": 0, "max": 5}
)
plt.show()
print(f"χ² = {model_global.𝜒2:.2f}")
print("\nGlobal optimizer finds better fit!")

## 5. Comparing Different Methods

Let's compare multiple optimization methods on the same problem:


In [ ]:
# Use multi-peak data for comparison
df_multi = generate_multi_peak_data(n_points=200, seed=42)

# Define two-peak model
def two_gaussians(x, A1, c1, w1, A2, c2, w2, B):
    """Sum of two Gaussians plus baseline"""
    from ezfit import gaussian
    return gaussian(x, A1, c1, w1) + gaussian(x, A2, c2, w2) + B

# Define parameter bounds
params = {
    "A1": {"value": 7.0, "min": 0, "max": 15},
    "c1": {"value": 7.0, "min": 5, "max": 9},
    "w1": {"value": 2.0, "min": 0.5, "max": 5},
    "A2": {"value": 5.0, "min": 0, "max": 15},
    "c2": {"value": 12.0, "min": 10, "max": 14},
    "w2": {"value": 3.0, "min": 0.5, "max": 5},
    "B": {"value": 0.5, "min": 0, "max": 2}
}

methods = ["curve_fit", "minimize", "differential_evolution", "dual_annealing"]
results = {}

for method in methods:
    try:
        print(f"\n=== {method.upper()} ===")
        model, ax, _ = df_multi.fit(
            two_gaussians, "x", "y", "yerr",
            method=method,
            fit_kwargs={"seed": 42} if method in ["differential_evolution", "dual_annealing"] else {},
            **params
        )
        results[method] = {"model": model, "chi2": model.𝜒2}
        print(f"χ² = {model.𝜒2:.2f}")
        plt.close()  # Close plot to save memory
    except Exception as e:
        print(f"Failed: {e}")
        results[method] = None

# Compare results
print("\n" + "="*50)
print("COMPARISON SUMMARY")
print("="*50)
for method, result in results.items():
    if result:
        print(f"{method:25s}: χ² = {result['chi2']:.2f}")
    else:
        print(f"{method:25s}: Failed")

## 6. Visualizing the Best Fit

Let's plot the best result:


In [ ]:
# Find best method
best_method = min([m for m, r in results.items() if r],
                  key=lambda m: results[m]['chi2'])

print(f"Best method: {best_method}")

# Re-fit and plot
model_best, ax_best, ax_res_best = df_multi.fit(
    two_gaussians, "x", "y", "yerr",
    method=best_method,
    fit_kwargs={"seed": 42} if best_method in ["differential_evolution", "dual_annealing"] else {},
    **params
)

plt.show()
print(model_best)

## 7. When to Use Which Method?

### Decision Tree:

1. **Simple, well-behaved functions** → `curve_fit` (default)

   - Fast, reliable for most cases
   - Good initial guesses usually sufficient

2. **Many local minima or complex surfaces** → `differential_evolution` or `dual_annealing`

   - Global optimizers that search entire parameter space
   - Slower but more robust

3. **Need uncertainty estimates** → `curve_fit` or `emcee` (MCMC)

   - `curve_fit` provides covariance matrix
   - `emcee` provides full posterior distributions

4. **Linear models** → `ridge`, `lasso`, or `bayesian_ridge`
   - Specialized methods for linear regression
   - Can handle regularization

## Summary

In this tutorial, you learned:

1. ✅ When initial guesses matter
2. ✅ The difference between local and global optimizers
3. ✅ How to compare different methods
4. ✅ When to use each optimization method

**Next Steps:**

- Try the advanced tutorial to learn about MCMC and constraints
- Experiment with your own complex models
- Read about parameter constraints in the documentation
